In [10]:
from neo4j import GraphDatabase
import requests
import os
from dotenv import load_dotenv
import pandas as pd

## Download csv file
Download from https://fixturedownload.com/results/nfl-2024

In [11]:
# URL of the CSV file
url = "https://fixturedownload.com/download/nfl-2024-UTC.csv"

# Create the data directory if it doesn't exist
if not os.path.exists('./data'):
    os.makedirs('./data')

# File path where we'll save the CSV
file_path = './data/nfl-2024-UTC.csv'

try:
    # Send GET request to the URL
    response = requests.get(url)
    
    # Raise an exception for bad status codes
    response.raise_for_status()
    
    # Write the content to a file
    with open(file_path, 'wb') as f:
        f.write(response.content)
        
    print(f"Successfully downloaded and saved the file to {file_path}")
    
except requests.exceptions.RequestException as e:
    print(f"An error occurred while downloading the file: {e}")
except IOError as e:
    print(f"An error occurred while saving the file: {e}")

Successfully downloaded and saved the file to ./data/nfl-2024-UTC.csv


## Connect to Neo4j

In [7]:
load_dotenv()

uri = os.getenv('NEO4J_URI')
user = os.getenv('NEO4J_USER')
password = os.getenv('NEO4J_PASSWORD')
database = os.getenv('NEO4J_DATABASE')

In [9]:
driver = GraphDatabase.driver(uri, auth=(user, password), database=database)

## Load data to Neo4j

In [14]:
games_df = pd.read_csv('./data/nfl-2024-UTC.csv')
games_df

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,16,1,06/09/2024 00:20,GEHA Field at Arrowhead Stadium,Kansas City Chiefs,Baltimore Ravens,27 - 20
1,15,1,07/09/2024 00:15,Arena Corinthians,Philadelphia Eagles,Green Bay Packers,34 - 29
2,7,1,08/09/2024 17:00,Mercedes-Benz Stadium,Atlanta Falcons,Pittsburgh Steelers,10 - 18
3,8,1,08/09/2024 17:00,Highmark Stadium,Buffalo Bills,Arizona Cardinals,34 - 28
4,9,1,08/09/2024 17:00,Soldier Field,Chicago Bears,Tennessee Titans,24 - 17
...,...,...,...,...,...,...,...
280,281,Divisional Round,19/01/2025 20:00,Lincoln Financial Field,Philadelphia Eagles,Los Angeles Rams,28 - 22
281,282,Divisional Round,19/01/2025 23:30,Highmark Stadium,Buffalo Bills,Baltimore Ravens,27 - 25
282,283,Conference Champs,26/01/2025 20:00,Lincoln Financial Field,Philadelphia Eagles,Washington Commanders,55 - 23
283,284,Conference Champs,26/01/2025 23:30,GEHA Field at Arrowhead Stadium,Kansas City Chiefs,Buffalo Bills,32 - 29


In [16]:
load_query = """
    UNWIND $data AS row
    MERGE (h:Team {name:row.`Home Team`})
    MERGE (a:Team {name:row.`Away Team`})
    MERGE (h)-[:PLAYS]->(a)
    """

result = driver.execute_query(load_query, {"data": games_df[['Home Team', 'Away Team']].to_dict("records")})
print(result.summary.counters)

{'_contains_updates': True, 'labels_added': 32, 'relationships_created': 279, 'nodes_created': 32, 'properties_set': 32}
